In [46]:
from requests_html import AsyncHTMLSession
import asyncio
import json
import numpy as np
import pandas as pd
import time


In [47]:
tours = []

for year in [2021, 2022, 2023, 2024]:
    with open(f'data/wtt/tournaments{year}.json') as f:
        tours.extend(json.load(f)[0]['rows'])

In [48]:
tours = [
    tour for tour in tours
    if (
        (
            tour['EventType'].startswith('WTT')
            and tour['EventType'] != 'WTTC Finals Regional Qualifications'
        )
        or tour['EventType'].startswith('ITTF')
        or tour['EventType'] == 'Olympic Games'
        or tour['EventType'] == 'World Championships'
    )
]
tours = [
    tour for tour in tours
    if (
        'Youth' not in tour['EventType']
        and 'Exhibition' not in tour['EventType']
    )
]

In [49]:
tf = pd.DataFrame(tours)
tf.drop_duplicates(subset=['EventId'], keep='last', inplace=True)
tf.EventType.value_counts()

EventType
WTT Feeder                 47
WTT Contender              34
WTT Star Contender         12
WTT Champions              10
ITTF International Open    10
WTT Grand Smash             5
WTTC                        4
Olympic Games               3
WTT Finals                  3
WTT Cup Finals              2
World Championships         2
Name: count, dtype: int64

In [50]:
tf.drop(inplace=True, columns=['PageLink', 'FromStartDate', 'ToStartDate', 'FromEndDate', 'ToEndDate', 'ShowInCalendar', 'Type', 'EventCode', 'Comments', 'EventDateChangeId', 'Tags'])


In [51]:
rows = tf[
    tf.EventName.str.contains('U11', case=False)
    | tf.EventName.str.contains('U13', case=False)
    | tf.EventName.str.contains('U15', case=False)
    | tf.EventName.str.contains('U17', case=False)
    | tf.EventName.str.contains('U19', case=False)
    | tf.EventName.str.contains('Senior', case=False)
    | tf.EventName.str.contains('Youth', case=False)
    | tf.EventName.str.contains('Junior', case=False)
    | tf.EventName.str.contains('Qualification', case=False)
    | tf.EventName.str.contains('Parkinson', case=False)
    | tf.EventName.str.contains('Veteran', case=False)
]
rows

,EventName,EventType,EventTypeId,Country,City,ContinentCode,Subcontinent,StartDateTime,EndDateTime,EventId,Event_Tier_Name
16,ITTF World Youth Championships,ITTF International Open,80,Portugal,Vila Nova De Gaia,europe,southern europe,2021-12-02T00:00:00,2021-12-08T00:00:00,2511,NaN
18,World Parkinson Table Tennis Championships,World Championships,2,Germany,Berlin,europe,western europe,2021-09-09T00:00:00,2021-09-11T00:00:00,2514,NaN
19,Central American Veteran Championships,World Championships,2,El Salvador,El Salvador,americas,central america,2021-09-14T00:00:00,2021-09-18T00:00:00,2515,NaN
69,2023 ITTF-Oceania Olympic Team & Mixed Doubles...,Olympic Games,85,Australia,None,oceania,australasia,2023-09-02T00:00:00,2023-09-03T00:00:00,2717,NaN
86,2023 South American U11 & U13 Championships,ITTF International Open,80,Paraguay,Asunción,americas,south america,2023-05-09T00:00:00,2023-05-14T00:00:00,2756,NaN
128,ITTF Americas Olympic Qualification event,ITTF International Open,80,Peru,Lima,americas,south america,2024-05-14T00:00:00,2024-05-18T00:00:00,2935,NaN
132,ITTF Africa Olympic Qualification event,ITTF International Open,80,Rwanda,Kigali,africa,east africa,2024-05-16T00:00:00,2024-05-18T00:00:00,2946,NaN


In [53]:
tf.drop(tf[tf.EventId==2608].index, inplace=True)

In [54]:
tf.drop(rows.index, inplace=True)

In [55]:
tf.to_csv('data/tournaments_wtt.tsv', sep='\t', index=False)

In [56]:
tf = pd.read_csv('data/tournaments_wtt.tsv', sep='\t')